## Schema of our building
Inserer image

In [360]:
#[!bldg](builidng.pdf)

## Dimensions of the building

Here, we compute the dimensions of our 2 rooms:
we have chosen to take into account only one wall in contact with the rest of the chalet and one wall with the 2m² window in contact with the outside

In [361]:
l_br=3 #lenght of the bedroom
L_br=4 #width of the bedroom
l_d= 3 #lenght of the dressing
L_d= 2 #width of the dressing
height= 3   #height under ceiling

S_br= l_br*L_br  #total surface bedroom
S_d= l_d*L_d    #total surface dressing

S_glass= 2   # surface of the glass taken into account in the study

S_wall_3=l_d*height  # surface of the wall between the dressing and the bedroom
S_wall_4=L_d*height  # surface of the wall of the dressing
S_wall_2= L_br*height # surface de droite 
S_wall_1= l_br*height - S_glass #surface du haut
 



h_in = 8 #convection coefficient for inside, W/m² K;
h_out= 25 #convection coefficient for outside, W/m² K;
T_out = 10   #temperature outside, °C;
T_in = 20   #temperature inside, °C;
T_i0 = T_out +273 #temp out, K;


Phi0 = 4550
Phia = 760

σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant


## Desciption of the building
inserer photo adri

## Thermo-physical properties

In [362]:
#Sapin
wood = {'Conductivity': 0.06,
            'Density': 450,
            'Specific heat': 2300,
            'Width':0.2 ,
            'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4
             }

whool = {'Conductivity': 0.034,
              'Density': 40, 
              'Specific heat': 1030,
              'Width':0.25 ,
              'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4 }

glass = {'Conductivity': 1 ,
         'Density': 2500,
         'Specific heat': 0.8,
         'Width':0.04 ,
         'Surface': S_glass}

air = {'Conductivity': 0.024 ,
         'Density': 1.293,
         'Specific heat': 1005,
         'Width':0.1 ,
         'Surface': S_glass }   
          

## Radiative properties

In [363]:
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

## Thermal circuit
Inserer photo


## Thermal conductances

### Conduction

In [364]:
G_1 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_2 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_3 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_4 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_10= wood['Conductivity'] / wood['Width'] * S_wall_2
G_17= wood['Conductivity'] / wood['Width'] * S_wall_3
G_14= wood['Conductivity'] / wood['Width'] * S_wall_4
G_13= glass['Conductivity'] / glass['Width'] * S_glass


### Convection

In [365]:
G_0 = h_out * S_wall_1
G_5= h_in * S_wall_1
G_11= h_in * S_wall_2
G_16= h_in * S_wall_3
G_15= h_in * S_wall_4
G_6 = h_in * S_wall_3
G_9 = h_in * S_glass
G_12= h_out * S_glass


### Long wave radiation

In [366]:
#Long Wave radiation
Tm = T_in + 273   # K, mean temp for radiative exchange
Fwg = glass['Surface']  / wood['Surface']    # view factor wall-glass

#Between window and wall_3
G7_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G7_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G7_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
G_7 = 1 / (1 / G7_1 + 1 / G7_12 + 1 / G7_2) 

#Between wall_1 and wall_3
G8_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G8_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G8_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * S_wall_1 
G_8 = 1 / (1 / G8_1 + 1 / G8_12 + 1 / G8_2)

## Thermal capacities

In [367]:
C_1 = wood['Density'] * wood['Specific heat'] * wood['Surface'] * wood['Width']      #wood capacity wall 1
C_2 = whool['Density'] * whool['Specific heat'] * whool['Surface'] * whool['Width']  #whool capacity wall 1

## System of Algebraic Differential Equations (DAE)

### Incidence matrix A

Photo Matrice A

In [368]:
import numpy as np
nq = 18                         # n° of branches
n0 = 13                         # n° of nodes
A = np.zeros([nq,n0])             # n° of branches X n° of nodes
A[0, 0] = 1                     # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1        # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1        # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1        # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1        # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1        # branch 5: node 4 -> node 5
A[6, 5], A[6, 6] =  1,-1        # branch 6: node 6 -> node 5
A[7, 6], A[7, 7] =  1,-1        # branch 7: node 7 -> node 6
A[8, 4], A[8, 6] = -1, 1        # branch 8: node 4 -> node 6
A[9, 5], A[9, 7] =  1,-1        # branch 9: node 7 -> node 5
A[10, 8] = 1                    # branch 10: -> node 8
A[11, 5], A[11, 8] = 1, -1      # branch 11: node 8 -> node 5
A[12, 9] = 1                    # branch 12: -> node 9
A[13, 7], A[13, 9] = 1, -1      # branch 13: node 9 -> node 7
A[14, 12] = 1                   # branch 14: -> node 12
A[15, 11], A[15, 12] = 1, -1    # branch 15: node 12 -> node 11  
A[16, 10], A[16, 11] = 1, -1    # branch 16: node 11 -> node 10
A[17, 6], A[17, 10] = 1, -1     # branch 17: node 10 -> node 6
print(A)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.]]


### Conductance matrix G

Photo matrice G

In [369]:
G = np.diag(np.hstack([G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17]))
print(G)

[[175.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           2.1          0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           2.1          0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.952        0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.952
    0.           0.           0.           0.           0.
   

### Capacity Matrix C

Photo matrice C

In [370]:
C = np.diag(np.hstack([0,C_1,0,C_2,0,0,0,0,0,0,0,0,0]))

### Temperature vector Teta

In [371]:
Teta = np.zeros(n0)

### Temperature source vector b

In [372]:
b = np.zeros(nq)
b[0] = T_out
b[10] = T_in
b[12] = T_out
b[14] = T_in

### Heat flow source vector f

In [373]:
f = np.zeros(n0)
f[0] = Phi0
f[9] = Phia

### Heat flow vector q

In [374]:
q = np.zeros(nq)

### Computation of Teta

In [375]:
Teta = np.linalg.inv(A.T @ G @ A) @ (A.T @ G @ b + f)
print(f'θ = {Teta} °C')

θ = [25.98293032 24.56045686 23.13798341 20.00017431 16.86236522 16.81118936
 16.82143255 15.89443027 16.92644758 15.54721514 18.04690434 18.09285953
 18.16179232] °C


### Computation of q

In [376]:
q = G @ (-A @ Teta+b)
print(f'q = {q} W')

q = [-4.54701281e+03  2.98719426e+00  2.98719426e+00  2.98719426e+00
  2.98719426e+00  2.86584770e+00  7.37509080e-01 -2.69261131e+00
  1.21346559e-01 -1.46681455e+01  1.10647887e+01  1.10647887e+01
 -7.77360757e+02 -1.73607568e+01  3.30877383e+00  3.30877383e+00
  3.30877383e+00  3.30877383e+00] W
